In [1]:
import os
import glob
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, utils
from torch.utils.data import DataLoader, Dataset
import torch.autograd as autograd
import matplotlib.pyplot as plt
import numpy as np
from pytorch_fid import fid_score
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


ModuleNotFoundError: No module named 'pytorch_fid'

### 1. Data Preparation & Data Preprocessing:    - Score: 10 points

In [ ]:
pth_to_imgs = "img_align_celeba"
imgs = glob.glob(os.path.join(pth_to_imgs, "*"))

In [ ]:
class CelebADataset(Dataset):
    def __init__(self, img_paths, transform=None):
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img

### 이미지 전처리

In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
dataset = CelebADataset(imgs, transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

data_iter = iter(dataloader)
images = next(data_iter)

plt.figure(figsize=(8, 8))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(((images[i].permute(1, 2, 0) + 1) / 2).numpy())
    plt.axis("off")
plt.show()

### 2. Generator and Discriminator: - Score: 20 points

In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim=100):
        super(Generator, self).__init__()
        self.init_size = 4
        self.l1 = nn.Sequential(nn.Linear(z_dim, 256 * self.init_size * self.init_size))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(256),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),

            nn.ConvTranspose2d(32, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 256, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.utils.spectral_norm(nn.Conv2d(3, 32, 4, 2, 1, bias=False)),
            nn.LeakyReLU(0.2, inplace=True),

            nn.utils.spectral_norm(nn.Conv2d(32, 64, 4, 2, 1, bias=False)),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            nn.utils.spectral_norm(nn.Conv2d(64, 128, 4, 2, 1, bias=False)),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.utils.spectral_norm(nn.Conv2d(128, 256, 4, 2, 1, bias=False)),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.model(x)
        return out.view(-1, 1)


In [ ]:
def gradient_penalty(D, real_data, fake_data, device):
    alpha = torch.rand(real_data.size(0), 1, 1, 1, device=device)
    interpolates = (alpha * real_data + (1 - alpha) * fake_data).requires_grad_(True)
    d_interpolates = D(interpolates)
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=torch.ones_like(d_interpolates),
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    return ((gradients.norm(2, dim=1) - 1) ** 2).mean()

### 3. Training: - Score: 10 points

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

z_dim = 100
G = Generator(z_dim).to(device)
D = Discriminator().to(device)

if os.path.isfile("saved_models/generator.pth") :
    G.load_state_dict(torch.load('saved_models/generator.pth', map_location=device))
    D.load_state_dict(torch.load('saved_models/discriminator.pth', map_location=device))
    G.eval()
    D.eval()
    print("Generator와 Discriminator 기존 모델 불러오기")

else :
    optimizer_G = optim.Adam(G.parameters(), lr=0.0001, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(D.parameters(), lr=0.00005, betas=(0.5, 0.999))
    lambda_gp = 10
    num_epochs = 50

In [ ]:
if not os.path.isfile("saved_models/generator.pth"):
    os.makedirs('saved_models', exist_ok=True)
    os.makedirs('training_logs', exist_ok=True)
    os.makedirs('generated_samples', exist_ok=True)

    d_loss_values, g_loss_values = [], []

    for epoch in range(num_epochs):
        for i, real_imgs in enumerate(dataloader):
            real_imgs = real_imgs.to(device)
            batch_size = real_imgs.size(0)

            z = torch.randn(batch_size, z_dim, 1, 1, device=device)
            fake_imgs = G(z).detach()
            real_validity = D(real_imgs)
            fake_validity = D(fake_imgs)
            gp = gradient_penalty(D, real_imgs, fake_imgs, device)

            d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gp
            optimizer_D.zero_grad()
            d_loss.backward(retain_graph=True)
            optimizer_D.step()

            if i % 5 == 0:
                z = torch.randn(batch_size, z_dim, 1, 1, device=device)
                fake_imgs = G(z)
                fake_validity = D(fake_imgs)
                g_loss = -torch.mean(fake_validity)
                optimizer_G.zero_grad()
                g_loss.backward()
                optimizer_G.step()

            d_loss_values.append(d_loss.item())
            g_loss_values.append(g_loss.item())

            if i % 50 == 0:
                with torch.no_grad():
                    sample_z = torch.randn(16, z_dim, 1, 1, device=device)
                    generated_imgs = G(sample_z).cpu()
                    grid = utils.make_grid(generated_imgs, nrow=4, normalize=True)
                    utils.save_image(grid, f'generated_samples/epoch_{epoch+1}_step_{i}.png')

                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(dataloader)}], "
                      f"D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")
        torch.save(G.state_dict(), f"saved_models/generator_epoch_{epoch+1}.pth")
        torch.save(D.state_dict(), f"saved_models/discriminator_epoch_{epoch+1}.pth")
        print(f"Models saved for epoch {epoch+1}")

    torch.save(G.state_dict(), f"saved_models/generator.pth")
    torch.save(D.state_dict(), f"saved_models/discriminator.pth")
    print(f"Models saved.")

### 4. Evaluation: - Score: 10 points

In [ ]:
def save_real_images(dataloader, num_samples=1000, save_dir='real_images_subset'):
    os.makedirs(save_dir, exist_ok=True)
    count = 0
    for imgs in dataloader:
        for img in imgs:
            img = ((img + 1) / 2).clamp(0, 1)
            utils.save_image(img, os.path.join(save_dir, f"real_{count}.png"))
            count += 1
            if count >= num_samples:
                return

os.makedirs('real_images_subset', exist_ok=True)
if len(os.listdir('real_images_subset')) < 1000:
    save_real_images(dataloader, num_samples=1000, save_dir='real_images_subset')
    print("Saved 1000 real images for FID calculation.")
else:
    print("Many Images.")

In [ ]:
def save_generated_images(G, z_dim=100, num_images=1000, batch_size=64, save_dir='generated_images_fid'):
    os.makedirs(save_dir, exist_ok=True)
    G.eval()
    with torch.no_grad():
        for i in range(0, num_images, batch_size):
            current_batch_size = min(batch_size, num_images - i)
            z = torch.randn(current_batch_size, z_dim, 1, 1, device=device)
            fake_imgs = G(z).cpu()
            fake_imgs = (fake_imgs * 0.5) + 0.5
            for j in range(fake_imgs.size(0)):
                utils.save_image(fake_imgs[j], os.path.join(save_dir, f"fake_{i+j}.png"))
    print(f"Saved {num_images} generated images to {save_dir}")

save_generated_images(G, z_dim=z_dim, num_images=1000, batch_size=64, save_dir='generated_images_fid')

In [ ]:


def compute_fid(real_dir, fake_dir):
    paths = [real_dir, fake_dir]
    fid_value = fid_score.calculate_fid_given_paths(
        paths,
        batch_size=128,
        device=device,
        dims=2048
    )
    print(f"FID: {fid_value}")

compute_fid(real_dir='real_images_subset', fake_dir='generated_images_fid')

In [ ]:
def visualize_generated_images(G, z_dim=100, num_images=64):
    G.eval()
    with torch.no_grad():
        sample_z = torch.randn(num_images, z_dim, 1, 1, device=device)
        generated_imgs = G(sample_z).cpu()
        generated_imgs = (generated_imgs * 0.5) + 0.5

    plt.figure(figsize=(8,8))
    grid = utils.make_grid(generated_imgs, nrow=8, normalize=False)
    plt.imshow(grid.permute(1, 2, 0).numpy())
    plt.axis("off")
    plt.show()

visualize_generated_images(G, z_dim=z_dim, num_images=64)

In [ ]:
def compare_real_fake(real_dir='real_images_subset', fake_dir='generated_images_fid'):
    real_imgs = sorted(glob.glob(os.path.join(real_dir, '*.png')))[:16]
    fake_imgs = sorted(glob.glob(os.path.join(fake_dir, '*.png')))[:16]

    plt.figure(figsize=(16, 8))
    for i in range(16):
        plt.subplot(4, 8, i + 1)
        img = Image.open(real_imgs[i]).convert("RGB")
        img = np.array(img)
        plt.imshow(img)
        plt.axis("off")
        if i == 7:
            plt.text(-10, 32, 'Real Images', fontsize=12, color='red')
        plt.subplot(4, 8, i + 1 + 16)
        img = Image.open(fake_imgs[i]).convert("RGB")
        img = np.array(img)
        plt.imshow(img)
        plt.axis("off")
        if i == 7:
            plt.text(-10, 32, 'Generated Images', fontsize=12, color='blue')
    plt.show()

compare_real_fake(real_dir='real_images_subset', fake_dir='generated_images_fid')

### 5. Latent Space Exploration

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import utils

def slerp(val, low, high):
    omega = torch.acos(torch.clamp(torch.dot(low.flatten(), high.flatten()) /
                                   (torch.norm(low) * torch.norm(high)), -1, 1))
    so = torch.sin(omega)
    if so == 0:
        return (low + high) / 2
    return (torch.sin((1.0 - val) * omega) / so) * low + (torch.sin(val * omega) / so) * high

def interpolate_latent_space_slerp(G, z_dim=100, steps=8, device='cpu'):
    G.eval()
    with torch.no_grad():
        z1 = torch.randn(1, z_dim, 1, 1, device=device)
        z2 = torch.randn(1, z_dim, 1, 1, device=device)
        alphas = np.linspace(0, 1, steps)
        interpolated_imgs = []

        for alpha in alphas:
            z = slerp(alpha, z1, z2)
            img = G(z).cpu()
            interpolated_imgs.append(img[0])
        interpolated_imgs = torch.stack(interpolated_imgs)

        grid = utils.make_grid(interpolated_imgs, nrow=steps, normalize=True)

        plt.figure(figsize=(steps * 2, 2))
        plt.imshow(grid.permute(1, 2, 0).numpy())
        plt.title("Latent Space Slerp Interpolation")
        plt.axis("off")
        plt.show()

interpolate_latent_space_slerp(G, z_dim=z_dim, steps=8, device=device)


이번 과제는 GAN을 이용해 CelebA 데이터셋을 학습 시키고, 이를 통해 가상의 얼굴 이미지를 생성하는것이 목적입니다.
Inference 부분에서는 전체 과정에 대한 상세한 설명과 함께, 모델 아키텍처, 학습 과정, 평가 결과 등을 작성하였습니다.

## 1. 모델 아키텍처

### a. Generator
Generator는 랜덤한 잠재 벡터
𝑧
z를 입력으로 받아 실제 데이터 분포와 유사한 이미지를 생성하는 역할을 수행합니다. 이 과정은 아래와 같은 단계로 이루어집니다:

입력:

𝑧
z: 크기 100의 랜덤 잠재 벡터
입력된
𝑧
z는 노이즈를 포함하며, 이를 기반으로 이미지를 생성합니다.
ConvTranspose2d 레이어:

역합성곱(Transpose Convolution): 잠재 벡터를 4x4x256의 텐서로 확장하고, 이를 반복적으로 업샘플링하여 최종적으로 64x64 크기의 이미지를 생성합니다.
업샘플링 과정에서 채널 크기를 점차 줄이며 다음과 같은 해상도를 만듭니다:
4
×
4
4×4 →
8
×
8
8×8 →
16
×
16
16×16 →
32
×
32
32×32 →
64
×
64
64×64.
BatchNorm2d:

각 ConvTranspose2d 레이어 이후 배치 정규화를 적용하여 학습 안정성과 수렴 속도를 향상시킵니다.
활성화 함수:

중간 레이어에는 ReLU 활성화 함수가 사용되어 음수를 제거하고 비선형성을 도입합니다.
최종 레이어에는 Tanh 활성화 함수가 사용되어 출력을 [-1, 1] 범위로 정규화합니다.
출력:

RGB 채널의 이미지를 생성하며 출력 크기는
64
×
64
×
3
64×64×3입니다.
역전파 가능:

ConvTranspose2d와 BatchNorm2d의 조합으로 Generator는 역전파가 효율적으로 이루어지도록 설계되어 있습니다.
b. Discriminator
Discriminator는 Generator가 생성한 가짜 이미지와 실제 이미지를 구분하는 역할을 합니다. 이미지의 진위를 판별하며, 아래와 같은 구조로 설계되었습니다:

입력:

𝑥
x:
64
×
64
×
3
64×64×3 크기의 RGB 이미지
실제 이미지와 가짜 이미지를 모두 입력받습니다.
Conv2d 레이어:

합성곱: 입력 이미지를 64x64에서 32x32, 16x16, 8x8, 4x4로 점진적으로 다운샘플링하며, 채널 수는 32 → 64 → 128 → 256으로 증가합니다.
다운샘플링을 통해 고수준의 특징을 학습합니다.
BatchNorm2d:

각 Conv2d 레이어 이후 배치 정규화를 적용하여 학습 안정성과 성능을 향상시킵니다.
활성화 함수:

LeakyReLU 활성화 함수가 사용됩니다. 이 함수는 음수 기울기를 도입하여 0이 아닌 작은 값을 반환하며, 정보 손실을 줄이고 모델의 표현력을 향상시킵니다.
최종 레이어:

Conv2d로 4x4의 텐서를 1x1로 축소하며, Sigmoid 활성화 함수를 사용해 출력을 [0, 1] 범위로 정규화합니다.
출력값은 이미지가 실제일 확률(0은 가짜, 1은 진짜)을 나타냅니다.
출력:

스칼라 값(이미지 진위 여부 확률)을 반환합니다.
추가 설명
Generator와 Discriminator의 관계: Generator는 Discriminator를 속이기 위해 더 실제와 유사한 이미지를 생성하려고 하며, Discriminator는 이 이미지를 구별하려고 학습합니다. 이 과정은 경쟁적(Adversarial)인 학습 과정으로, 두 모델이 서로 발전하면서 균형에 도달합니다.
활성화 함수 선택:
Generator의 최종 레이어에서 Tanh는 출력값을 [-1, 1]로 정규화하여 데이터 분포에 맞게 이미지를 생성하도록 돕습니다.
Discriminator의 최종 레이어에서 Sigmoid는 확률로 변환해 진위 여부를 학습합니다.

## 2. 학습 과정

### a. 데이터 준비 및 전처리

1. **데이터셋:** CelebA 데이터셋을 사용하여 얼굴 이미지 200,000장을 포함
2. **전처리:**
- 2.1. **크기 조정:** 모든 이미지를 64x64 픽셀로 리사이즈
- 2.2. **텐서 변환:** 이미지를 텐서로 변환
- 2.3. **정규화:** 픽셀 값을 [-1, 1] 범위로 정규화 (\( \text{Normalize}((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) \))
- 2.4. **DataLoader:** 배치 크기 64, 셔플링 적용하여 데이터 로드

### b. 손실 함수 및 옵티마이저

3. **손실 함수:** Binary Cross Entropy Loss (BCE Loss)
- 3.1. **Discriminator:** 실제 이미지와 가짜 이미지를 분류
- 3.2. **Generator:** Discriminator를 속이려는 목적으로 사용
4. **옵티마이저:** Adam 옵티마이저 사용
- 4.1. **학습률:** 0.0002
- 4.2. **베타 값:** (0.5, 0.999)

### c. 레이블 스무딩

- 5.1. **진짜 라벨:** 1 대신 0.9로 설정하여 Discriminator의 자신이 진짜 사람이라는 것을 인식하는것을 방지하기 위해 사용
- 5.2. **가짜 라벨:** 0으로 설정

### d. 학습 단계

- **에포크 수:** 50
- **훈련 단계:**
  1. **Discriminator 학습:**
     - 실제 이미지와 가짜 이미지를 통해 Discriminator의 손실 계산
     - 합산된 손실을 역전파하여 Discriminator의 가중치 업데이트
  2. **Generator 학습:**
     - 가짜 이미지를 통해 Discriminator의 출력을 받아 Generator의 손실 계산 (진짜로 속이려는 목표)
     - 손실을 역전파하여 Generator의 가중치 업데이트

### e. 모델 저장

- **모델 저장:** 모든 에포크가 완료된 후, Generator와 Discriminator의 가중치를 저장하여 추후 평가 및 재사용이 가능하도록 함

## 3. 아쉬운 점과 향후 개선 방향

1. **FID 점수**: 현재 FID 점수는 아쉬운 수준입니다. 이는 생성된 이미지의 품질과 다양성이 실제 이미지와 상당한 차이가 있음을 의미하고 있습니다. 더 많은 에포크로 학습하고 모델 구조를 개선하면 더 좋은 결과를 얻을 수 있을 것 같습니다.

2. **학습 시간**: 컴퓨팅 자원의 한계로 충분한 에포크 수로 학습하지 못한 것이 아쉽습니다. 현재 FID 점수를 보면 모델이 충분히 수렴하지 못했다는 것을 알 수 있습니다. GPU 환경에서 더 긴 시간 학습하면 더 나은 결과를 얻을 수 있을 것 같다고 생각됩니다.

3. **이미지 품질**: 생성된 이미지들이 아직 많이 흐릿하고 세부 디테일이 부족합니다. StyleGAN과 같은 최신 아키텍처를 적용하면 더 좋은 품질의 이미지를 생성할 수 있을 것 같다고 생각됩니다.